In [3]:
from Juliet_DB_loader import TestCase, Folder, SubFolder, sessionmaker, engine
from sqlalchemy import func
from sqlalchemy.orm import sessionmaker
import re

Session = sessionmaker(bind=engine)
session = Session()


The below code can be used to verify a specific test cases's content and line number before any modifications are made


In [ ]:
test_cases = session.query(TestCase).filter_by(id=1000).first()
line_content = dict()
lines = test_cases.file_content.split('\n')
multiline_comment = False
for line_num, line in enumerate(lines, start=1):
    line_content[line_num] = line

# Print the dictionary vertically
for line_num, line in line_content.items():
    print(f"{line_num}: {line}")

Some checks I want to run is to see if my idea of holding the sting then updating the vulnerability line number will work. I think that I need to check that no comments exist on the line of the vulnerability location. I'd also like to generally print out a bunch of the vulnerability line locations, I will probably need to do this in conjunction with the linedict above to get the lines numbered. 


In [34]:
test_case = session.query(TestCase).filter_by(id=10000).first()
line_content = dict()
lines = test_case.file_content.split('\n')
vulnerable_line = int(test_case.vulnerability_location)

print(f"Vulnerable line: {vulnerable_line}")
print(f"Total lines in file: {len(lines)}")

for line_num, line in enumerate(lines, start=1):
    line_content[line_num] = line    

# Check if vulnerable_line is a valid key in line_content
if vulnerable_line in line_content:
    print(f"{vulnerable_line}: {line_content[vulnerable_line]}")
else:
    print(f"Vulnerable line {vulnerable_line} is not in the file content.")

Vulnerable line: 43
Total lines in file: 133
43:         strcpy(data, source);


## Lex and Yacc Parser Method
I'm going to mess around with a simple parser to see if it can properly strip whitespace rather than the faulty regular expressions I was using

In [6]:
import ply.lex as lex
import ply.yacc as yacc


# Define tokens
tokens = ['COMMENT', 'CODE']

# Define rules
t_COMMENT = r'(/\*([^*]|[\r\n]|(\*+([^*/]|[\r\n])))*\*+/)|(//.*)'
t_CODE = r'[^\n]+'

# Define lexer
lexer = lex.lex()

# Define parser
def p_code(p):
    '''code : code COMMENT
            | code CODE
            | COMMENT
            | CODE'''
    if len(p) == 2:
        p[0] = p[1]
    elif len(p) == 3:
        p[0] = p[1] + p[2]
def p_error(p):
    print("Syntax error in input!")
    
parser = yacc.yacc(write_tables=False)

def strip_comments_and_preserve_line_numbers(code):
    parsed_code = parser.parse(code)
    return parsed_code
c_code = session.query(TestCase).filter_by(id=60000).first()


stripped_code = strip_comments_and_preserve_line_numbers(c_code)
print(stripped_code)

KeyError: '__file__'